In [2]:
from google.colab import drive

drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import os
from tqdm import tqdm

In [4]:
from IPython.display import Image, clear_output  # to display images
#from utils.google_utils import gdrive_download  # to download models/datasets

#Analisis de las imágenes

In [ ]:
ruta = "/content/drive/MyDrive/TFM/PennFudanPed/Yolo"
#Numero de personas de cada clase
n_pedW = 0
n_pedS = 0

#Numero de imagenes que contienen al menos un objeto de la clase
n_imgW = 0
n_imgS = 0

#Lista imagenes con personas paradas
list_imgS = []

for i in tqdm(os.listdir(ruta)):
  walking = False
  standing = False
  yolo_ruta = os.path.join(ruta,i)
  #print(yolo_ruta)
  with open('%s' % yolo_ruta) as fichero:
       linea = fichero.readline() 
       #print(linea)
       #contador = 1
       persona = 0
       while linea:
         clase = linea[0]
         #print(clase)
         if clase == "0" and not walking:
           walking = True
           n_pedW += 1
           n_imgW += 1
         elif clase == "1" and not standing:
           standing = True
           n_pedS += 1
           n_imgS += 1
           list_imgS.append(i)
         elif clase == "0":
           n_pedW += 1
         elif clase == "1":
           n_pedS += 1
         linea = fichero.readline() 

print("\nExisten ", n_pedW, " personas andando y ", n_pedS, " personas paradas, habiendo ", n_imgW, 
      " imagenes con personas andando y ", n_imgS, " imagenes con personas paradas")
print(list_imgS)  

#Yolo v5

In [ ]:
!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
%pip install -qr requirements.txt  # install

import torch
import utils
display = utils.notebook_init()  # checks

#Una clase

In [ ]:
%cat /content/drive/MyDrive/TFM/PennFudanPed/TFM_pedestrian_Yolov5_one_class/data.yaml

In [ ]:
# train yolov5s on custom data for 100 epochs
# time its performance
%%time
%cd /content/yolov5/
!python train.py --img 416 --batch 16 --epochs 50 --data /content/drive/MyDrive/TFM/PennFudanPed/TFM_pedestrian_Yolov5_one_class/data.yaml --cfg ./models/yolov5l.yaml --weights yolov5l.pt --name yolov5l_results  --cache

In [ ]:
# Start tensorboard
# Launch after you have started training
# logs save in the folder "runs"
%load_ext tensorboard
%tensorboard --logdir runs

In [ ]:
# first, display our ground truth data
print("GROUND TRUTH TEST DATA:")
Image(filename='/content/yolov5/runs/train/yolov5l_results/train_batch0.jpg', width=900)

In [ ]:
# first, display our ground truth data
print("Prediction TEST DATA:")
Image(filename='/content/yolov5/runs/train/yolov5l_results/val_batch0_pred.jpg', width=900)

In [ ]:
%cd /content/yolov5/
!python val.py --weights /content/yolov5/runs/train/yolov5l_results3/weights/best.pt --data /content/drive/MyDrive/TFM/PennFudanPed/TFM_pedestrian_Yolov5_one_class/data.yaml --img 416 --iou 0.65 --half --task test

# Yolov7

In [ ]:
# Download YOLOv7 repository and install requirements
%cd content
!git clone https://github.com/WongKinYiu/yolov7
%cd yolov7
!pip install -r requirements.txt

In [ ]:
%cat /content/drive/MyDrive/TFM/PennFudanPed/TFM_pedestrian_one_class/data.yaml

In [ ]:
# run this cell to begin training
%%time
%cd /content/yolov7
!python train.py --batch 16 --cfg cfg/training/yolov7.yaml --epochs 55 --data /content/drive/MyDrive/TFM/PennFudanPed/TFM_pedestrian_Yolov7_one_class/data.yaml  --weights 'yolov7.pt' --device 0 --name yolov7_results  --cache

In [ ]:
# first, display our ground truth data
print("GROUND TRUTH TEST DATA:")
Image(filename='/content/yolov5/yolov7/runs/train/yolov7_results/train_batch0.jpg', width=900)

In [ ]:
# first, display our ground truth data
print("Prediction TEST DATA:")
Image(filename='/content/yolov5/yolov7/runs/train/yolov7_results/test_batch0_pred.jpg', width=900)